
> 본 archive 파일은 프로젝트 진행 과정의 '시행착오'를 기록한 연구 노트입니다.
> 최종 결과물이 아니며, 의도적인 에러나 폐기된 로직이 포함되어 있어 정상적으로 실행되지 않을 수 있습니다.

In [ ]:
import pandas as pd
import numpy as np
import gc
import os

# 1900만 행 데이터를 한 번에 로드할 때 Out of memory 발생
# 대용량 처리를 위한 Chunking 및 1차 전처리 파이프라인

INPUT_FILE = 'train.csv'
OUTPUT_FILE = '1차_train.feather'
CHUNK_SIZE = 500000
TARGET = 'nins'

# 학습에 사용할 기본 피처 리스트 정의
FEATURES = [
    'appr_temp', 'ceiling', 'cloud_b', 'dew_point', 'precip_1h', 'pressure',
    'temp_b', 'uv_idx', 'vis', 'cloud_a', 'humidity', 'rain', 'snow', 'temp_a',
    'coord1', 'coord2', 'time', 'pv_id',
    'real_feel_temp_shade', 'temp_max', 'temp_min', 'real_feel_temp',
    'wind_gust_spd', 'U_a', 'V_a', 'U_b', 'V_b' # 풍향/풍속은 벡터로 변환하여 사용
]

# 벡터 변환 전 원본 컬럼 포함
BASE_FEATURES = [f for f in FEATURES if f not in ['U_a', 'V_a', 'U_b', 'V_b']] + ['wind_dir_a', 'wind_spd_a', 'wind_dir_b', 'wind_spd_b']

def process_data_chunk(chunk_df):
    # 메모리 효율을 위해 카테고리형 및 float32로 다운캐스팅
    chunk_df['pv_id'] = chunk_df['pv_id'].astype('category')
    
    for col in chunk_df.select_dtypes(include=['float64']).columns:
        chunk_df[col] = chunk_df[col].astype('float32')
    for col in chunk_df.select_dtypes(include=['int64']).columns:
        chunk_df[col] = chunk_df[col].astype('int32')

    chunk_df['time'] = pd.to_datetime(chunk_df['time'], errors='coerce')
    
    # 필요한 컬럼만 필터링하여 반환
    final_features = [f for f in BASE_FEATURES if f in chunk_df.columns]
    if TARGET in chunk_df.columns:
        return chunk_df[final_features + [TARGET]]
    return chunk_df[final_features]

# --- 1. Chunking 로드 실행 ---
print(f"[{INPUT_FILE}] 분할 로드(Chunking) 시작...")
processed_chunks = []

for i, chunk in enumerate(pd.read_csv(INPUT_FILE, chunksize=CHUNK_SIZE, engine='python')):
    print(f"Chunk {i+1} 처리 중...")
    processed_chunk = process_data_chunk(chunk)
    processed_chunks.append(processed_chunk)

# 분할 처리된 데이터 병합
df = pd.concat(processed_chunks, ignore_index=True)
del processed_chunks
gc.collect()
print("데이터 로드 및 병합 완료.")

# --- 2. 풍향/풍속 벡터(U/V) 변환 ---
# 풍향(도)은 연속성이 없으므로(0도=360도), 벡터 성분으로 변환하여 학습 효율 증대
print("풍향/풍속 벡터 변환 수행...")
def convert_wind_to_vectors(df, prefix):
    dir_col = f'wind_dir_{prefix}'
    spd_col = f'wind_spd_{prefix}'
    if dir_col in df.columns and spd_col in df.columns:
        rad = df[dir_col] * np.pi / 180.0
        df[f'U_{prefix}'] = df[spd_col] * np.cos(rad)
        df[f'V_{prefix}'] = df[spd_col] * np.sin(rad)
    return df

df = convert_wind_to_vectors(df, 'a')
df = convert_wind_to_vectors(df, 'b')

# --- 3. 하이브리드 보간 (Hybrid Interpolation) ---
print("보간 수행 (Linear + Ffill/Bfill)...")
# 발전소별 시간순 정렬 필수
df.sort_values(by=['pv_id', 'time'], inplace=True)
df.reset_index(drop=True, inplace=True)

# (1) 연속형 변수: 선형(Linear) 보간 적용
linear_cols = [
    'appr_temp', 'dew_point', 'pressure', 'temp_b', 'temp_a', 
    'real_feel_temp_shade', 'temp_max', 'temp_min', 'real_feel_temp',
    'humidity', 'wind_gust_spd', 'U_a', 'V_a', 'U_b', 'V_b'
]
linear_cols = [c for c in linear_cols if c in df.columns]

# (2) 상태형 변수: 앞뒤 값 채우기(Ffill/Bfill) 적용
ffill_cols = ['ceiling', 'cloud_b', 'precip_1h', 'uv_idx', 'vis', 'cloud_a', 'rain', 'snow']
ffill_cols = [c for c in ffill_cols if c in df.columns]

g = df.groupby('pv_id')

if linear_cols:
    df[linear_cols] = g[linear_cols].transform(lambda x: x.interpolate(method='linear', limit_direction='both'))

if ffill_cols:
    df[ffill_cols] = g[ffill_cols].transform(lambda x: x.bfill().ffill())

# 잔여 결측치(야간 시간대 등)는 0으로 처리
df.fillna(0, inplace=True)

print(f"전처리 완료. 저장: {OUTPUT_FILE}")